In [148]:

from detector import Detector
import glob
from torchsummary import summary
import matplotlib.pyplot as plt
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
import json
from PIL import Image
from IPython.display import display
import random

test_text = '''
Do you have a problem that you are struggling to solve? Why don't you ask your friends for advice? When people ask for advice on solving a problem, often times they speak to more than one person. This is because different views are better for figuring out a tough problem, many opinions are better than one, and other people may have experienced a problem like yours and may be able to help you in making better decisions.
'''

# AI Text Image Recognition

Is there some similarity to Malware detection, where it's common to translate malware code to bytes and then images, which are then fed into NN/DL networks? Let's test a bit.

In [139]:
ai = json.load(open("E:\\Python_Projects\\PrismAI\\PrismAI\\src\\outputs\\generator_1_mistralai_Mistral-7B-v0.1_100_outputs.json", encoding='utf8'))
ai = ai + json.load(open("E:\\Python_Projects\\PrismAI\\PrismAI\\src\\outputs\\generator_1_daryl149_llama-2-7b-chat-hf_100_outputs.json", encoding='utf8'))
ai = ai + json.load(open("E:\\Python_Projects\\PrismAI\\PrismAI\\src\\outputs\\generator_1_gpt2_100_outputs.json", encoding='utf8'))

human = json.load(open("E:\\Python_Projects\\PrismAI\\PrismAI\\src\\outputs\\generator_0_mistralai_Mistral-7B-v0.1_100_outputs.json", encoding='utf8'))
human = human + json.load(open("E:\\Python_Projects\\PrismAI\\PrismAI\\src\\outputs\\generator_0_daryl149_llama-2-7b-chat-hf_100_outputs.json", encoding='utf8'))
human = human + json.load(open("E:\\Python_Projects\\PrismAI\\PrismAI\\src\\outputs\\generator_0_gpt2_100_outputs.json", encoding='utf8'))
print(len(ai))
print(len(human))

300
300


## Vis as Image


In [140]:
def output_to_image(detector_outputs):
    for output in detector_outputs:
        meta = output['metadata']
        height = meta['sample_rate']
        width = meta['sample_sequence_length']
        results = output['ensemble_results'][0]['sample_results']
        
        image = Image.new('RGBA', (100, 20), color=(0,0,0,255))
        pixels = image.load()

        x = 1
        offset_x = x
        y = 1

        for result in results:
            steps = result['model_outputs']['steps']
            steps.sort(key= lambda x: x['step'])

            for step in steps:                
                r = 255
                g = 255
                b = 255
                
                all_probs = step['top_k_probs']
                all_probs.sort(reverse=True)
                other_probs = all_probs[1:]
                
                # mid
                pixels[x, y] = (r, g, b, int(step['target_prob'] * 255))
                # right
                pixels[x+1, y] = (r, g, b, int(other_probs[0] * 255))
                # left
                pixels[x-1, y] = (r, g, b, int(other_probs[1] * 255))
                # down
                pixels[x, y-1] = (r, g, b, int(other_probs[2] * 255))
                # up
                pixels[x, y+1] = (r, g, b, int(other_probs[3] * 255))
                
                x += 3
            
            offset_x = int((offset_x % 2 == 1))
            x = offset_x
            y += 3
        display(image)

def draw_image(width, height):
    image = Image.new('RGBA', (width, height))
    pixels = image.load()
    # iterate over all pixels
    for y in range(height):
        for x in range(width):
            r = random.randint(0, 255)
            g = random.randint(0, 255)
            b = random.randint(0, 255)
            a = random.randint(0, 1)
            pixels[x, y] = (r, g, b, int(a * 255))  # Set the pixel RGBA values
    return image
    

In [141]:
output_to_image(human[:2])
output_to_image(ai[:2])


# CNN

Let's try some CNN detection on the steps

## 2D CNN

Let's start by creating 2d arrays with x channels per "pixel"

In [142]:
def create_dataset(detector_outputs, label):
    data = []
    labels = []
    
    for output in detector_outputs:
        results = output['ensemble_results'][0]['sample_results']
        rows = []

        for result in results:
            steps = result['model_outputs']['steps']
            steps.sort(key= lambda x: x['step'])
            columns = []
            for step in steps:
                channels = [step['target_prob']]
                top_k_probs = step['top_k_probs']
                top_k_probs.sort()
                channels = channels + top_k_probs
                columns.append(channels)
                
            rows.append(columns)
          
        data.append(rows)
        labels.append(label)  
    data = np.array(data)
    labels = np.array(labels)
    
    return data, labels

In [143]:
data_ai, labels_ai = create_dataset(ai, 1)
data_hu, labels_hu = create_dataset(human, 0)
data = np.concatenate((data_ai, data_hu), axis=0)
labels = np.concatenate((labels_hu, labels_ai))
print(len(labels))

# Convert numpy arrays to tensors
# We need: [batch_size, channels, height, width]
# data_tensor = torch.tensor(data, dtype=torch.float32).permute(0, 2, 1).unsqueeze(-2)
data_tensor = torch.tensor(data, dtype=torch.float32).permute(0, 3, 1, 2)
print(data_tensor.shape)
labels_tensor = torch.tensor(labels, dtype=torch.long)

print(data_tensor[:1])
print(labels_tensor[:1])

600
torch.Size([600, 6, 6, 24])
tensor([[[[3.9993e-01, 9.6910e-01, 7.8637e-01, 5.3396e-03, 9.9930e-01,
           5.7553e-01, 9.9835e-01, 8.9255e-02, 5.3373e-02, 6.4691e-01,
           8.9303e-01, 4.7557e-03, 3.0601e-02, 9.9994e-01, 3.3034e-01,
           4.3163e-01, 5.0898e-01, 3.1150e-02, 4.1730e-01, 5.7956e-03,
           7.9316e-01, 1.5728e-02, 6.6157e-01, 9.6784e-01],
          [5.4341e-01, 2.0752e-02, 3.5485e-01, 8.5465e-02, 2.8040e-04,
           6.5149e-02, 9.3025e-01, 8.1803e-01, 4.4634e-01, 8.6016e-01,
           4.0067e-01, 5.4798e-01, 9.7516e-01, 6.6756e-01, 5.7694e-01,
           9.2365e-01, 6.7627e-01, 8.3559e-01, 2.8993e-01, 9.9992e-01,
           2.5412e-01, 7.1047e-02, 3.0996e-01, 9.9353e-01],
          [3.8078e-02, 5.5823e-01, 7.6015e-01, 2.1506e-01, 8.6310e-01,
           9.9965e-01, 4.0253e-01, 5.8665e-01, 2.4741e-02, 9.8571e-01,
           7.1991e-01, 8.6504e-01, 9.8645e-01, 4.4676e-02, 6.5583e-01,
           2.4257e-01, 9.4451e-01, 6.4040e-01, 5.0905e-01, 6.2726e-

# Train, Test Split

In [144]:
X_train, X_test, y_train, y_test = train_test_split(data_tensor, labels_tensor, test_size=0.2, random_state=42)
print("x_train shape",X_train.shape)
print("x_test shape",X_test.shape)
print("y_train shape",y_train.shape)
print("y_test shape",y_test.shape)
print(y_test[:20])

x_train shape torch.Size([480, 6, 6, 24])
x_test shape torch.Size([120, 6, 6, 24])
y_train shape torch.Size([480])
y_test shape torch.Size([120])
tensor([0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0])


# Model


In [175]:
model = nn.Sequential(
    nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(1,3), padding=(0, 1)),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=(1,2), stride=(1, 2)),
    nn.Dropout(p=0.25),

    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(1,3), padding=(0,1)),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=(1,2), stride=(1,2)),
    nn.Dropout(p=0.25),

    nn.Flatten(),
    nn.Linear(in_features=36, out_features=32),
    nn.ReLU(),
    nn.Dropout(p=0.4),

    nn.Linear(in_features=32, out_features=1),
    nn.Sigmoid()
)

print(model)

Sequential(
  (0): Conv2d(6, 16, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
  (3): Dropout(p=0.25, inplace=False)
  (4): Conv2d(16, 32, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
  (5): ReLU()
  (6): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
  (7): Dropout(p=0.25, inplace=False)
  (8): Flatten(start_dim=1, end_dim=-1)
  (9): Linear(in_features=36, out_features=64, bias=True)
  (10): ReLU()
  (11): Dropout(p=0.4, inplace=False)
  (12): Linear(in_features=64, out_features=1, bias=True)
  (13): Sigmoid()
)


In [176]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [177]:
# Training the model
num_epochs = 75
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    model.train()  # Set the model to training mode
    for inputs, labels in zip(X_train, y_train):
        optimizer.zero_grad()
        outputs = model(inputs)
        
        if len(labels.size()) == 0:
            labels = labels.unsqueeze(0).expand(32, 1).float()
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        # Calculate training accuracy
        predicted = outputs > 0.5  # Assuming threshold of 0.5 for binary classification
        total += labels.size(0)
        correct += (predicted == labels.byte()).sum().item()
    
    # Print training loss and accuracy
    train_loss = running_loss / len(y_train)
    train_accuracy = correct / total
    print(f"Epoch {epoch+1}, Training Loss: {train_loss:0.5f}, Training Accuracy: {train_accuracy:0.5f}")
    
    # Test loss and accuracy calculation
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for test_inputs, test_labels in zip(X_test, y_test):
            test_outputs = model(test_inputs)
            if len(test_labels.size()) == 0:
                test_labels = test_labels.unsqueeze(0).expand(32, 1).float()
            test_loss += criterion(test_outputs, test_labels).item()
            test_predicted = test_outputs > 0.5
            test_total += test_labels.size(0)
            test_correct += (test_predicted == test_labels.byte()).sum().item()
    
    # Print test loss and accuracy
    test_loss /= len(y_test)
    test_accuracy = test_correct / test_total
    print(f"Epoch {epoch+1}, Test Loss: {test_loss:0.5f}, Test Accuracy: {test_accuracy:0.5f}\n")


ValueError: Using a target size (torch.Size([64, 1])) that is different to the input size (torch.Size([32, 1])) is deprecated. Please ensure they have the same size.